## 四种文档处理链

- stuff
- refine

### **StuffChain**
最常见的文档链，将文档直接塞进Prompt中，为LLM回答问题提供上下文资料，适合小文档的场景
#### **代码示例**

In [11]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain_ollama import ChatOllama

loader = PyPDFLoader("../03-ChatDoc/files/file.pdf")
print(loader)

prompt_template = """用一句话对以下文字做简要总结：
{text}
总结："""

prompt = PromptTemplate.from_template(prompt_template)

llm = ChatOllama(model="llama3.1:8b")

llm_chain = LLMChain(
  llm=llm,
  prompt=prompt
)

stuff_chain = StuffDocumentsChain(
  llm_chain=llm_chain,
  document_variable_name="text",
  verbose=True
)

docs = loader.load()
result = stuff_chain.run(docs)
print(result)


Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)




> Entering new StuffDocumentsChain chain...

> Finished chain.
该合同主要内容包括：合同主体、教学服务内容、教学安排、教学费用的支付方式和退费政策等。


### 使用已经封装好的 load_summarize_chain

In [12]:
from langchain.chains.summarize import load_summarize_chain

load_sum_chain = load_summarize_chain(
  llm=llm,
  chain_type="stuff",
  verbose=True
)

print(load_sum_chain.run(docs))



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"电吉他教学服务合同  合同编号：_______________   
甲⽅（学员/家⻓）：   
姓名：___________________________   
身份证号：_______________________   
联系⽅式：_______________________ 
⼄⽅（教师）：   
姓名：___________________________   
身份证号：_______________________   
联系⽅式：_______________________ 
第⼀条 合同主体   
1. 甲⽅为学员本⼈或法定监护⼈，具有签订本合同的权利和能⼒，需按时⽀付教学费⽤并配合教学
安排。   
2. ⼄⽅ _____________________ 专业教师，承诺提供⾼质量的⼀对⼀教学服务。   
第⼆条 教学服务内容 
1. 教学乐器：电吉他（含必要配件，如效果器、拨⽚等）。   
2. 教学⽬标：通过系统教学，帮助学员掌握电吉他基本演奏技巧、⾳阶、和弦、节奏及即兴演奏能
⼒，提升⾳乐表现⼒。   
3. 教学⽅式：采⽤⼀对⼀⾯授⽅式，结合理论讲解、示范演奏与实践练习。   
第三条 教学安排  
1. 服务地点：⼄⽅指定教学场所（详细地址：__________________________________________），
如需变更需提前3⽇协商⼀致。   
2. 单次服务时间：   
   - 每次课程时⻓：_________⼩时（建议45分钟-1⼩时）   
   - 课程周期：⾃_________年_________⽉_________⽇起⾄_________年_________⽉_________⽇
⽌，每周_________次。   
3. 课程阶段：根据学员⽔平动态调整，初期侧重基础技巧，后期逐步增加复杂曲⽬与即兴创作。   
 ⻚码： /1 2

第四条 教学费⽤ 
1. 收费标准：⼈⺠币_________元/课时（